In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/Shareddrives/Skripsi/dataset/split.zip'

Streaming output truncated to the last 5000 lines.
  inflating: split/reptiles/train/crocodile/nile_crocodile_s_001696.png  
  inflating: split/reptiles/train/crocodile/crocodylus_porosus_s_000144.png  
  inflating: split/reptiles/train/crocodile/nile_crocodile_s_000215.png  
  inflating: split/reptiles/train/crocodile/crocodylus_porosus_s_000265.png  
  inflating: split/reptiles/train/crocodile/crocodylus_porosus_s_000461.png  
  inflating: split/reptiles/train/crocodile/crocodylus_porosus_s_000048.png  
  inflating: split/reptiles/train/crocodile/crocodylus_niloticus_s_000557.png  
  inflating: split/reptiles/train/crocodile/nile_crocodile_s_000990.png  
  inflating: split/reptiles/train/crocodile/crocodile_s_000791.png  
  inflating: split/reptiles/train/crocodile/crocodile_s_000028.png  
  inflating: split/reptiles/train/crocodile/crocodile_s_003028.png  
  inflating: split/reptiles/train/crocodile/nile_crocodile_s_000037.png  
  inflating: split/reptiles/train/crocodile/african_cr

In [21]:
val_df=pd.read_csv("/content/drive/Shareddrives/Skripsi/notebooks/tes/validation.csv")

In [22]:
val_df=val_df.drop('Unnamed: 0',axis=1)

In [23]:
SEED=42
DIM=32

val_img_gen = ImageDataGenerator(
    rescale=1./255,
)
val_gen=val_img_gen.flow_from_dataframe(
    val_df,
    x_col="file_path",
    target_size=(DIM, DIM),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False,
)

Found 6000 validated image filenames.


In [24]:
import keras.backend as K
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
      recall = true_positives / (possible_positives + K.epsilon())
      return recall

    def precision(y_true, y_pred):
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
      precision = true_positives / (predicted_positives + K.epsilon())
      return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [25]:
coarse_model=keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/coarse.h5',custom_objects={'f1_score':f1_score})
single_model=keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/all.h5',custom_objects={'f1_score':f1_score})
fine_model=[
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/aquatic_mammals.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/fish.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/insects.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/large_carnivores.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/large_omnivores_and_herbivores.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/medium_mammals.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/non-insect_invertebrates.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/people.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/reptiles.h5',custom_objects={'f1_score':f1_score}),
    keras.models.load_model('/content/drive/Shareddrives/Skripsi/model/f1/small_mammals.h5',custom_objects={'f1_score':f1_score}),
]


In [26]:
coarse_label_dict={
    'aquatic_mammals': 0,
    'fish': 1,
    'insects': 2,
    'large_carnivores': 3,
    'large_omnivores_and_herbivores': 4,
    'medium_mammals': 5,
    'non-insect_invertebrates': 6,
    'people': 7,
    'reptiles': 8,
    'small_mammals': 9
 }

fine_label=[
    {'beaver': 0, 'dolphin': 1, 'otter': 2, 'seal': 3, 'whale': 4},
    {'aquarium_fish': 0, 'flatfish': 1, 'ray': 2, 'shark': 3, 'trout': 4},
    {'bee': 0, 'beetle': 1, 'butterfly': 2, 'caterpillar': 3, 'cockroach': 4},
    {'bear': 0, 'leopard': 1, 'lion': 2, 'tiger': 3, 'wolf': 4},
    {'camel': 0, 'cattle': 1, 'chimpanzee': 2, 'elephant': 3, 'kangaroo': 4},
    {'fox': 0, 'porcupine': 1, 'possum': 2, 'raccoon': 3, 'skunk': 4},
    {'crab': 0, 'lobster': 1, 'snail': 2, 'spider': 3, 'worm': 4},
    {'baby': 0, 'boy': 1, 'girl': 2, 'man': 3, 'woman': 4},
    {'crocodile': 0, 'dinosaur': 1, 'lizard': 2, 'snake': 3, 'turtle': 4},
    {'hamster': 0, 'mouse': 1, 'rabbit': 2, 'shrew': 3, 'squirrel': 4},

]

In [27]:
# def generate_df():
#     res_coarse=coarse_model.predict(val_gen)
#     res_coarse_idx_arr=[]
#     res_coarse_arr=[]

#     for i in res_coarse:
#         idx=i.argmax(axis=0)
#         res_coarse_idx_arr.append(idx)
#         res_coarse_arr.append(list(coarse_label_dict.keys())[idx])
    
#     new_df=val_df.copy()
#     new_df['predicted_label_coarse']=res_coarse_arr
#     new_df['predicted_label_index_coarse']=res_coarse_idx_arr
#     return new_df

In [28]:
import time
import tracemalloc
start=time.time()
tracemalloc.start()


In [29]:
res_coarse=coarse_model.predict(val_gen)
res_coarse_idx_arr=[]
res_coarse_arr=[]

for i in res_coarse:
    idx=i.argmax(axis=0)
    res_coarse_idx_arr.append(idx)
    res_coarse_arr.append(list(coarse_label_dict.keys())[idx])

new_df=val_df.copy()
new_df['predicted_label_coarse']=res_coarse_arr
new_df['predicted_label_index_coarse']=res_coarse_idx_arr

188/188 [==============================] - 4s 22ms/step


In [30]:
def make_prediction(new_df):
    fine_arr=[]

    prediction_val_gen=val_img_gen.flow_from_dataframe(
            new_df,
            x_col="file_path",
            target_size=(DIM, DIM),
            color_mode='rgb',
            class_mode=None,
            batch_size=1,
            shuffle=False
        )

    for i in range(0, new_df.shape[0]): 
        coarse_index=new_df.iloc[i]['predicted_label_index_coarse']
        
        prediction=fine_model[coarse_index].predict(prediction_val_gen[i],  verbose=0)[0]
        prediction_argmax_index=prediction.argmax(axis=0)

        fine_dict=fine_label[coarse_index]
        fine_arr.append(list(fine_dict.keys())[prediction_argmax_index])

    return fine_arr


In [31]:
final_df=new_df.copy()

final_df['prediction_label_fine']=make_prediction(final_df)
finish=time.time()-start

Found 6000 validated image filenames.


In [32]:
print(tracemalloc.get_traced_memory())
 
# stopping the library
tracemalloc.stop()

(7173464, 19752384)


In [33]:
finish

650.8354942798615

In [16]:
finish/60

6.879788645108541

In [19]:
final_df

,file_path,label_fine,label_coarse,predicted_label_coarse,predicted_label_index_coarse,prediction_label_fine
0,./split/aquatic_mammals/val/otter/river_otter_...,otter,aquatic_mammals,non-insect_invertebrates,6,snail
1,./split/aquatic_mammals/val/otter/river_otter_...,otter,aquatic_mammals,insects,2,caterpillar
2,./split/aquatic_mammals/val/otter/river_otter_...,otter,aquatic_mammals,fish,1,trout
3,./split/aquatic_mammals/val/otter/river_otter_...,otter,aquatic_mammals,aquatic_mammals,0,otter
4,./split/aquatic_mammals/val/otter/river_otter_...,otter,aquatic_mammals,non-insect_invertebrates,6,snail
...,...,...,...,...,...,...
5995,./split/small_mammals/val/mouse/house_mouse_s_...,mouse,small_mammals,small_mammals,9,mouse
5996,./split/small_mammals/val/mouse/harvest_mouse_...,mouse,small_mammals,small_mammals,9,shrew
5997,./split/small_mammals/val/mouse/mus_musculus_s...,mouse,small_mammals,small_mammals,9,squirrel
5998,./split/small_mammals/val/mouse/mouse_s_000655...,mouse,small_mammals,aquatic_mammals,0,beaver


In [20]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print(classification_report(final_df['label_fine'], final_df['prediction_label_fine']))

print('accuracy_score:{}'.format(accuracy_score(final_df['label_fine'], final_df['prediction_label_fine'])))

print('F1-score:{}'.format(f1_score(final_df['label_fine'], final_df['prediction_label_fine'], average='weighted')))

               precision    recall  f1-score   support

aquarium_fish       0.70      0.82      0.75       120
         baby       0.45      0.38      0.41       120
         bear       0.55      0.30      0.39       120
       beaver       0.40      0.38      0.39       120
          bee       0.69      0.61      0.65       120
       beetle       0.69      0.62      0.65       120
          boy       0.37      0.53      0.44       120
    butterfly       0.51      0.63      0.56       120
        camel       0.56      0.57      0.56       120
  caterpillar       0.57      0.60      0.58       120
       cattle       0.52      0.56      0.54       120
   chimpanzee       0.66      0.64      0.65       120
    cockroach       0.76      0.79      0.78       120
         crab       0.48      0.54      0.51       120
    crocodile       0.48      0.47      0.47       120
     dinosaur       0.53      0.64      0.58       120
      dolphin       0.58      0.55      0.56       120
     elep